In [1]:
import re
import os
import numpy as np
import pandas as pd
from pandas._libs.tslibs import dtypes, timestamps
import subprocess
from functools import reduce
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
# Using matplotlib backend: QtAgg

In [2]:
def printRow(row):
    print("TIME:", row['system_time'])
    print(row['action'])
    print("==============")

system = "aurora"
if system == "aurora":
    stat_dir = "/home/gsd/andrelucena/statistics"
elif system == "slurm":
    stat_dir = "/projects/a97485/statistics"

model = "resnet50"
n_nodes = "4"
n_epochs = "2"
batch_size = "64"
save_every = "1"
log = "true"

stat_test = "eBPFs_subset_t2"
if system == "aurora":
    test_name = model + "_" + n_epochs + "_" + batch_size + "_" + save_every + "_" + log
elif system == "slurm":
    test_name = model + "_" + n_nodes + "_" + n_epochs + "_" + batch_size + "_" + save_every + "_" + log

full_test_path = stat_dir + "/" + stat_test + "/" + test_name
if system == "slurm":
    full_test_path += "/aurora03"

p = subprocess.Popen(["scp", f"{system}:{full_test_path}/icstat.bt", "./tmp/file"])
sts = os.waitpid(p.pid, 0)

data = None

with open("./tmp/file", "r") as fd:
    data = re.sub(r"\^C", "", fd.read())
    data = re.sub(r"%", "", data)
with open("./tmp/file", "w") as fd:
    fd.write(data)


df = pd.read_csv("./tmp/file", index_col=False, skiprows=[0,1,2,3])[:-2]
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%H:%M:%S.%f')

os.remove("./tmp/file")

print(df.head)

                 system_time                                    action  \
0 2024-11-05 01:27:19.453645                            Training begin   
1 2024-11-05 01:27:19.454282                          Training epoch 1   
3 2024-11-05 01:27:20.064135                Start Training Iteration 0   
4 2024-11-05 01:27:20.064876   Moving data to the same device as model   
5 2024-11-05 01:27:20.065609                          Computing output   

                duration  
0 0 days 00:00:00.000637  
1 0 days 00:00:00.609853  
3 0 days 00:00:00.000741  
4 0 days 00:00:00.000733  
5 0 days 00:00:00.904331  


In [7]:
def plotAction(df_param: pd.DataFrame, x, y, title):

    dt = df_param

    # plots action with time
    plt.figure()
    plt.plot(dt[x], dt[y], markersize = 5.0, marker = "+")
    plt.xlabel(x)
    plt.ylabel(y)
    plt.title(title)
    # plt.legend()
    plt.show()

In [8]:
plotAction(df, 'Timestamp', 'REFS', "Refs per time")

QStandardPaths: wrong permissions on runtime directory /run/user/0/, 0755 instead of 0700


In [ ]:
plotAction(df, 'Timestamp', 'MISSES', "Misses per time")

In [ ]:
plotAction(df, 'Timestamp', 'HIT', "Hits per time")